# QDAC Code Refactoring

## Import Libraries

In [2]:
import time
import json
import pyvisa
import numpy as np
import matplotlib.pyplot as plt
from time import sleep

from qcodes import Parameter, Measurement, Station, load_or_create_experiment, initialise_or_create_database_at
from qcodes.instrument.channel import ChannelList

from qstl_instruments.qstl_qdac2 import QSTL_QDac2
from qstl_instruments.qstl_nidaq import QSTL_NIDaq
from qstl_instruments.sg386 import SG386

from pyvisa.constants import StopBits, Parity

rm = pyvisa.ResourceManager()
rm.list_resources()

('ASRL1::INSTR',
 'ASRL3::INSTR',
 'ASRL5::INSTR',
 'ASRL6::INSTR',
 'ASRL8::INSTR',
 'ASRL9::INSTR',
 'ASRL10::INSTR',
 'ASRL20::INSTR',
 'ASRL21::INSTR',
 'ASRL22::INSTR',
 'ASRL23::INSTR',
 'ASRL24::INSTR',
 'ASRL25::INSTR',
 'ASRL26::INSTR',
 'ASRL27::INSTR',
 'GPIB0::27::INSTR')

## Instantiation of Instruments

In [3]:
## Dunker numbering
contacts = {"C1":1, "C2":2, "C3":3}
## SET gnd: 3, DQD gnd: 6
ai_chans = {"I1":"Dev2/ai0", "I2":"Dev2/ai1", "I3":"Dev2/ai2"}
initialise_or_create_database_at("./QDAC_IQ_Mod_Test.db")

qdac2 = QSTL_QDac2(
    name = "qdac2",
    address = "ASRL5::INSTR",
    ramp_rate = 1,
    i_threshold = 2e-9,
    v_limit = 0.5,
    contacts = contacts
)
station = Station(qdac2)
daq = QSTL_NIDaq(
    max_sampling_rate = int(1e6),
    gain = 1e8
)
srs_rf = SG386(
    name = "SRS_SG386",
    address = "GPIB0::27::INSTR"
)

qdac2.ramp_all_channels_to_zero()
qdac2.get_initial_voltages()

Connected to: QDevil QDAC-II (serial:368, firmware:13-1.57) in 0.09s
Connected to: Stanford Research Systems SG386 (serial:s/n004342, firmware:ver1.50.26) in 0.02s


{'C1': 0.1916136, 'C2': 0.1895836, 'C3': 0.0}

## QDAC2 Parameter Setup

In [14]:
## setup qdac 2 for the 2D sweep
qdac2.free_all_triggers()
qdac2.ext3.delay_s(0)
qdac2.v_limit = 1.2

device1 = "I1"
device2 = "I2"

slow_chans = ["C1"]
fast_chans = ["C2"]

slow_start = 0
slow_end = 0.2
slow_steps = 10

fast_start = 0
fast_end = 0.2
fast_steps = 10
fast_step_time_s = 0.20

qdac2.channels[0:23].dc_slew_rate_V_per_s('inf')

for i in [slow_start, slow_end, fast_start, fast_end]:
    qdac2.validate_voltages([i]) ## validate all the voltages

slow_vs = np.linspace(slow_start, slow_end, slow_steps, endpoint=True)
fast_vs = np.linspace(fast_start, fast_end, fast_steps, endpoint=True)

exp = load_or_create_experiment('2D sweep', "QDAC+NiDAQ_RF_test_100us_420MHz_LO_test_CH1_I_CH2_Q_I1_I_demod")
meas = Measurement(exp=exp, station=station)

Vslow = Parameter(name='Vslow', label=str(slow_chans), unit="V")
Vfast = Parameter(name="Vfast", label=str(fast_chans), unit="V")

I = Parameter(name= "I", label="I", unit="V")
Q = Parameter(name= "Q", label="Q", unit="V")

meas.register_parameter(Vfast)
meas.register_parameter(Vslow)
meas.register_parameter(I, setpoints = (Vfast, Vslow))
meas.register_parameter(Q, setpoints = (Vfast, Vslow))

## Measurement (Single Channel)

In [5]:
sweep_time = fast_steps * fast_step_time_s
samples_per_fast_scan_per_channel = fast_steps * int(fast_step_time_s * daq.max_sampling_rate/2)

arrangement = qdac2.arrange(
        contacts= {x: contacts[x] for x in fast_chans},
        output_triggers={'NIDAQ': 5}
)

sweep = arrangement.virtual_detune(
        contacts=tuple(fast_chans),
        start_V=(fast_start,) * len(fast_chans), ## a tuple with same length
        end_V=(fast_end,) * len(fast_chans),
        steps=fast_steps,
        step_trigger='NIDAQ',
        step_time_s = fast_step_time_s,
        repetitions=1
)

InitialConditions= qdac2.get_initial_voltages()

start_time = time.time()
with meas.run() as datasaver:
        loop_counter = 0
        datasaver.dataset.add_metadata(tag="Contacts", metadata=json.dumps(contacts))
        datasaver.dataset.add_metadata(tag="IC", metadata=json.dumps(InitialConditions))
        datasaver.dataset.add_metadata(tag="Sweep_params", metadata=json.dumps({"slow_chans": slow_chans, "slow_start":slow_start, "slow_end": slow_end, "fast_chans":fast_chans, "fast_start":fast_start, "fast_end":fast_end,"fast_step_time_s": fast_step_time_s} ))
        for slow_v in slow_vs:
                # sleep(0.100)
                qdac2.ramp_channels(slow_chans, [slow_v])
                result_0 = daq.read_triggered_voltage(sweep, ai_chans[device1], samples_per_fast_scan_per_channel, -1, +1, sweep_time+1)
                result_0 = daq.reshape_array(result_0, fast_steps)

                result_1 = daq.read_triggered_voltage(sweep, ai_chans[device2], samples_per_fast_scan_per_channel, -1, +1, sweep_time+1)
                result_1 = daq.reshape_array(result_1, fast_steps)

                amp = np.abs(result_0+1j*result_1)
                phase = np.angle(result_0+1j*result_1, deg=True)

                datasaver.add_result( (Vslow, [slow_v]*fast_steps), (Vfast, fast_vs), (I, result_0), (Q, result_1))
                loop_counter = loop_counter+1
                print(f'Time elapsed: {np.round(time.time()-start_time, 2)} sec. Loop finished: {loop_counter}/{slow_steps}.')
end_time = time.time()
print(f'Time elapsed: {np.round(end_time-start_time, 2)} sec.')

qdac2.channels[0:23].dc_slew_rate_V_per_s(1)

Starting experimental run with id: 153. 
Time elapsed: 2.39 sec. Loop finished: 1/10.
Time elapsed: 4.68 sec. Loop finished: 2/10.
Time elapsed: 6.99 sec. Loop finished: 3/10.
Time elapsed: 9.28 sec. Loop finished: 4/10.
Time elapsed: 11.57 sec. Loop finished: 5/10.
Time elapsed: 13.87 sec. Loop finished: 6/10.
Time elapsed: 16.17 sec. Loop finished: 7/10.
Time elapsed: 18.48 sec. Loop finished: 8/10.
Time elapsed: 20.77 sec. Loop finished: 9/10.
Time elapsed: 23.06 sec. Loop finished: 10/10.
Time elapsed: 23.07 sec.


## Measurement (Multi Channel)

In [15]:
sweep_time = fast_steps*fast_step_time_s
samples_per_fast_scan_per_channel = fast_steps*int(fast_step_time_s*daq.max_sampling_rate/2)

arrangement = qdac2.arrange(
        contacts= {x: contacts[x] for x in fast_chans},
        output_triggers={'NIDAQ': 5})

sweep = arrangement.virtual_detune(
        contacts=tuple(fast_chans),
        start_V=(fast_start,)*len(fast_chans), ## a tuple with same length
        end_V=(fast_end,)*len(fast_chans),
        steps=fast_steps,
        step_trigger='NIDAQ',
        step_time_s = fast_step_time_s,
        repetitions=1
)

InitialConditions = qdac2.get_initial_voltages()

start_time = time.time()
with meas.run() as datasaver:
        loop_counter = 0
        datasaver.dataset.add_metadata(tag="Contacts", metadata=json.dumps(contacts))
        datasaver.dataset.add_metadata(tag="IC", metadata=json.dumps(InitialConditions))
        datasaver.dataset.add_metadata(tag="Sweep_params", metadata=json.dumps({"slow_chans": slow_chans, "slow_start":slow_start, "slow_end": slow_end, "fast_chans":fast_chans, "fast_start":fast_start, "fast_end":fast_end,"fast_step_time_s": fast_step_time_s} ))
        for slow_v in slow_vs:
                qdac2.ramp_channels(slow_chans, [slow_v])
                result = daq.read_triggered_multi_channels(sweep, [ai_chans[device1], ai_chans[device2]], samples_per_fast_scan_per_channel, -1, +1, sweep_time+1)
                result_0 = daq.reshape_array(result[0,:], fast_steps)
                result_1 = daq.reshape_array(result[1,:], fast_steps)
                datasaver.add_result( (Vslow, [slow_v]*fast_steps), (Vfast, fast_vs), (I, result_0) , (Q, result_1))
                loop_counter = loop_counter+1
                print(f'Time elapsed: {np.round(time.time()-start_time, 2)} sec. Loop finished: {loop_counter}/{slow_steps}.')
end_time = time.time()
print(f'Time elapsed: {np.round(end_time-start_time, 2)} sec.')

qdac2.channels[0:23].dc_slew_rate_V_per_s(1)

Starting experimental run with id: 158. 
Time elapsed: 2.37 sec. Loop finished: 1/10.
Time elapsed: 4.53 sec. Loop finished: 2/10.
Time elapsed: 6.69 sec. Loop finished: 3/10.
Time elapsed: 8.84 sec. Loop finished: 4/10.
Time elapsed: 11.01 sec. Loop finished: 5/10.
Time elapsed: 13.19 sec. Loop finished: 6/10.
Time elapsed: 15.35 sec. Loop finished: 7/10.
Time elapsed: 17.51 sec. Loop finished: 8/10.
Time elapsed: 19.66 sec. Loop finished: 9/10.
Time elapsed: 21.8 sec. Loop finished: 10/10.
Time elapsed: 21.81 sec.


## SRS RF Generator Control

In [12]:
srs_rf.enable_RF("ON")
srs_rf.frequency(420e6)

In [16]:
srs_rf.enable_RF("OFF")